<a href="https://colab.research.google.com/github/nkikn/Facial-Expression-Recognition/blob/main/facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install kaggle
!pip install wandb torch torchvision pandas numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
# Download the dataset
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

 93% 266M/285M [00:00<00:00, 404MB/s]
100% 285M/285M [00:00<00:00, 445MB/s]


In [9]:
# Facial Expression Recognition - Simple CNN Baseline
# Notebook 1: Simple CNN Architecture

# Cell 1: Setup and Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import wandb
import os
from tqdm import tqdm
import time


In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Cell 2: Wandb Setup
# Initialize Wandb
wandb.login()

# Wandb configuration
config = {
    'model_name': 'test_overfit',
    'learning_rate': 0.001,
    'batch_size': 128,
    'epochs': 20,
    'optimizer': 'Adam',
    'loss_function': 'CrossEntropyLoss',
}

# Initialize wandb run
run = wandb.init(
    project="facial-expression-recognition",
    name="test_overfit",
    config=config,
)


Using device: cuda


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nkikn21 (nkikn21-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader


class My_Set(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx]['pixels']
        image = np.array([int(x) for x in pixels.split()]).reshape(48, 48)
        image = Image.fromarray(image.astype(np.uint8), mode='L')

        if self.transform:
            image = self.transform(image)

        label = self.data.iloc[idx]['emotion']
        return image, label


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),    # randomly flip image horizontally
    transforms.RandomRotation(10),        # randomly rotate by ±10 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # small translation
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),  # no augmentation on validation
])

from sklearn.model_selection import train_test_split

train_df = pd.read_csv('train.csv')
train_indices, val_indices = train_test_split(
    range(len(train_df)),
    test_size=0.2,
    stratify=train_df['emotion'],
    random_state=42
)



train_dataset = My_Set(train_df, transform=train_transform)
val_dataset = My_Set(train_df, transform=val_transform)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=2,
    # sampler=sampler
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=2
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Training samples: 28709
Validation samples: 28709


In [ ]:
class ResNetDropout(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.5):
        super().__init__()
        self.base = resnet18(pretrained=False)
        self.base.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.base.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.base.fc.in_features, num_classes)
        )

    def forward(self, x):
        return self.base(x)


In [ ]:

# Initialize model
model = ResNetDropout().to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


Total parameters: 2,360,647
Trainable parameters: 2,360,647


In [ ]:

# Log model info to wandb
wandb.log({"total_parameters": total_params, "trainable_parameters": trainable_params})

weight_decay = 1e-4  # small positive value to regularize weights

# Cell 5: Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=weight_decay)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)


In [ ]:

# Cell 6: Training Functions
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0   

    pbar = tqdm(train_loader, desc="Training")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({
            'Loss': f'{running_loss/len(train_loader):.4f}',
            'Acc': f'{100*correct/total:.2f}%'
        })

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total

    return epoch_loss, epoch_acc


In [16]:

# Validation function
def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100.0 * correct / total

    return epoch_loss, epoch_acc, all_preds, all_targets

In [17]:

# Cell 7: Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        start_time = time.time()

        # Training phase
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

        # Validation phase
        val_loss, val_acc, val_predictions, val_labels = validate_epoch(model, val_loader, criterion, device)

        # Update learning rate
        scheduler.step()

        # Calculate epoch time
        epoch_time = time.time() - start_time

        # Store metrics
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "epoch_time": epoch_time
        })

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_simple_cnn.pth')
            wandb.log({"best_val_accuracy": best_val_acc})

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"Epoch Time: {epoch_time:.2f}s")

        # Early stopping check (optional)
        if epoch > 20 and val_acc < best_val_acc - 5:  # Stop if val acc drops by 5%
            print("Early stopping triggered")
            break

    return train_losses, train_accuracies, val_losses, val_accuracies, val_predictions, val_labels


In [ ]:

# Cell 8: Train Model
print("Starting training...")
train_losses, train_accs, val_losses, val_accs, final_predictions, final_labels = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, config['epochs']
)


Starting training...

Epoch 1/30
----------


Training: 100%|██████████| 1000/1000 [00:06<00:00, 146.75it/s, Loss=0.7096, Acc=59.15%]


Train Loss: 0.7096, Train Acc: 59.15%
Val Loss: 0.5954, Val Acc: 73.75%
Epoch Time: 9.89s

Epoch 2/30
----------


Training: 100%|██████████| 1000/1000 [00:07<00:00, 138.47it/s, Loss=0.5651, Acc=71.45%]


Train Loss: 0.5651, Train Acc: 71.45%
Val Loss: 0.4444, Val Acc: 83.80%
Epoch Time: 9.77s

Epoch 3/30
----------


Training: 100%|██████████| 1000/1000 [00:06<00:00, 160.70it/s, Loss=0.4673, Acc=77.20%]


Train Loss: 0.4673, Train Acc: 77.20%
Val Loss: 0.5506, Val Acc: 70.40%
Epoch Time: 9.26s

Epoch 4/30
----------


Training: 100%|██████████| 1000/1000 [00:06<00:00, 146.93it/s, Loss=0.3762, Acc=82.90%]


Train Loss: 0.3762, Train Acc: 82.90%
Val Loss: 0.2482, Val Acc: 92.95%
Epoch Time: 9.29s

Epoch 5/30
----------


Training:  86%|████████▌ | 861/1000 [00:05<00:01, 131.04it/s, Loss=0.2370, Acc=88.53%]

In [ ]:

# Cell 9: Results Analysis and Visualization
# Plot training curves
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()


In [ ]:

# Confusion Matrix
plt.subplot(1, 3, 3)
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
cm = confusion_matrix(final_labels, final_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.savefig('simple_cnn_results.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

# Log plots to wandb
wandb.log({"training_curves": wandb.Image(plt)})

# Cell 10: Detailed Performance Analysis
# Classification Report
print("\nClassification Report:")
print(classification_report(final_labels, final_predictions, target_names=class_names))

# Calculate per-class accuracy
class_accuracies = {}
for i, class_name in enumerate(class_names):
    class_mask = np.array(final_labels) == i
    if np.sum(class_mask) > 0:
        class_acc = np.sum(np.array(final_predictions)[class_mask] == i) / np.sum(class_mask)
        class_accuracies[class_name] = class_acc * 100

print("\nPer-class Accuracies:")
for class_name, acc in class_accuracies.items():
    print(f"{class_name}: {acc:.2f}%")

# Log final metrics to wandb
final_metrics = {
    "final_train_accuracy": train_accs[-1],
    "final_val_accuracy": val_accs[-1],
    "best_val_accuracy": max(val_accs),
    "final_train_loss": train_losses[-1],
    "final_val_loss": val_losses[-1],
}

# Add per-class accuracies
for class_name, acc in class_accuracies.items():
    final_metrics[f"class_accuracy_{class_name.lower()}"] = acc

wandb.log(final_metrics)


In [ ]:

# # Cell 11: Model Summary and Insights
# print("\n" + "="*50)
# print("SIMPLE CNN MODEL SUMMARY")
# print("="*50)
# print(f"Architecture: {config['architecture']}")
# print(f"Total Parameters: {total_params:,}")
# print(f"Best Validation Accuracy: {max(val_accs):.2f}%")
# print(f"Final Validation Accuracy: {val_accs[-1]:.2f}%")
# print(f"Training Time: {len(train_accs)} epochs")

# print("\nKey Observations:")
# print("1. Simple architecture with only 2 conv layers")
# print("2. No regularization techniques applied")
# print("3. Basic architecture serves as baseline")
# print("4. Limited capacity may cause underfitting")
# print("5. Good starting point for progressive development")

# print("\nNext Steps:")
# print("1. Increase model depth (more conv layers)")
# print("2. Add regularization (BatchNorm, Dropout)")
# print("3. Experiment with different architectures")
# print("4. Apply data augmentation")


In [ ]:

# Cell 12: Save Model and Artifacts
# Save final model
torch.save(model.state_dict(), 'test_overfit.pth')

# Save model artifact to wandb
artifact = wandb.Artifact('simple_cnn_model', type='model')
artifact.add_file('test_overfit.pth')
wandb.log_artifact(artifact)

# Save training history
import pickle
history = {
    'train_losses': train_losses,
    'train_accuracies': train_accs,
    'val_losses': val_losses,
    'val_accuracies': val_accs,
    'config': config,
    'final_predictions': final_predictions,
    'final_labels': final_labels
}


with open('test_overfit_history.pkl', 'wb') as f:
    pickle.dump(history, f)

# Finish wandb run
wandb.finish()

print("\nTraining completed and logged to Wandb!")
print("Model saved as 'test_overfit.pth'")
print("Training history saved as 'test_overfit_history.pkl'")